In [2]:
import pandas as pd
import numpy as np
import requests
import keys
import json
from tqdm import tqdm

In [72]:
df = pd.read_csv('tm_events.csv')

In [73]:
venues = list(set(df.venue_name))

In [74]:
venues = [x.strip() for x in venues]

In [67]:
url = 'https://api.songkick.com/api/3.0/search/venues.json?'
success = []
fail = []
frame_list = []
for venue in tqdm(venues):
    try:
        req = requests.get(f'{url}query={venue}&apikey={keys.songkick}')
        x = json.loads(req.text)
        df = pd.DataFrame(x['resultsPage']['results']['venue'])
        frame_list.append(df)
        success.append(venue)
    except:
        fail.append(venue)
        


100%|██████████| 2779/2779 [26:50<00:00,  1.61it/s]


In [125]:
df = df[['displayName','id','capacity', 'street', 'city', 'zip', 'lat','lng','metroArea' ]]
df.rename(columns = {'displayName': 'venue_name', 'lat':'latitude', 'lng':'longitude'}, inplace=True)

In [126]:
df.reset_index(inplace=True)
df.rename(columns ={'index' : 'i'}, inplace=True)
df['metro_area'] = df.metroArea.apply(lambda x: x['displayName'])
df.drop(columns = ['city', 'metroArea', 'i'],inplace=True)

In [38]:
df.to_csv('sk_venues.csv')

In [2]:
df = pd.read_csv('sk_venues.csv', dtype='object')

In [3]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [4]:
venue_id = df[['venue_name', 'id']]

In [5]:
ids = list(set(venue_id.id))

In [6]:
url = 'https://api.songkick.com/api/3.0/venues/'
success = []
fail = []
frame_list = []
for id_ in tqdm(ids):
    try:
        req = requests.get(f'{url}{str(id_)}/calendar.json?apikey={keys.songkick}')
        x = json.loads(req.text)
        event_df = pd.DataFrame(x['resultsPage']['results']['event'])
        frame_list.append(event_df)
        success.append(id_)
    except:
        fail.append(id_)
        

100%|██████████| 11429/11429 [1:46:07<00:00,  2.19it/s] 


In [7]:
frames = frame_list.copy()

In [9]:
df = pd.concat(frames)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [54]:
df = pd.read_csv('sk_events.csv')

In [55]:
df['location'] = df.location.apply(lambda x: ast.literal_eval(x))
df['venue'] = df.venue.apply(lambda x: ast.literal_eval(x))
df['performance'] = df.performance.apply(lambda x: ast.literal_eval(x))
df['start'] = df.start.apply(lambda x: ast.literal_eval(x))

In [56]:
def try_artist(x, i):
    try:
        return x[i]['displayName']
    except:
        return np.nan
    
df.reset_index(inplace=True)
df.rename(columns={'index' : 'i'},inplace=True)
df['venue_id'] = df.venue.apply(lambda x: x['id'])
df['headliner'] = df.performance.apply(lambda x: try_artist(x, 0))
df['support'] = df.performance.apply(lambda x: try_artist(x, 1))
df['event_time'] = df.start.apply(lambda x: pd.to_datetime(x['datetime']))
df.drop(columns = ['series', 'end','status','uri', 'i','ageRestriction','venue', 'performance', 'location', 'displayName', 'start'], inplace=True)

In [53]:
df.head()

,Unnamed: 0,flaggedAsEnded,id,popularity,start,type,venue_id,headliner,support,event_time
0,0,False,37492754,0.013307,"{'date': '2019-08-08', 'datetime': '2019-08-08...",Concert,983,Reverend Horton Heat,The Delta Bombers,2019-08-08 19:00:00-06:00
1,1,False,38827049,0.000095,"{'date': '2019-08-10', 'datetime': '2019-08-10...",Concert,983,The Anchorage,Racist Kramer,2019-08-10 19:00:00-06:00
2,2,False,37091759,0.030854,"{'date': '2019-08-20', 'datetime': '2019-08-20...",Concert,983,King Gizzard & The Lizard Wizard,Stonefield,2019-08-20 19:00:00-06:00
3,3,False,38989431,0.009071,"{'date': '2019-08-24', 'datetime': '2019-08-24...",Concert,983,DJ Pauly D,NaN,2019-08-24 21:00:00-06:00
4,4,False,38991753,0.001657,"{'date': '2019-08-31', 'datetime': '2019-08-31...",Concert,983,Corduroy,NaN,2019-08-31 20:00:00-06:00


In [48]:
df

,Unnamed: 0,displayName,flaggedAsEnded,id,location,performance,popularity,start,type,venue,venue_id,headliner,support,event_time
0,0,Reverend Horton Heat with The Delta Bombers an...,False,37492754,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 71255334, 'displayName': 'Reverend Hor...",0.013307,"{'date': '2019-08-08', 'datetime': '2019-08-08...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Reverend Horton Heat,The Delta Bombers,2019-08-08 19:00:00-06:00
1,1,"The Anchorage, Racist Kramer, Problem Daughter...",False,38827049,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73571014, 'displayName': 'The Anchorag...",0.000095,"{'date': '2019-08-10', 'datetime': '2019-08-10...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,The Anchorage,Racist Kramer,2019-08-10 19:00:00-06:00
2,2,King Gizzard & The Lizard Wizard and Stonefiel...,False,37091759,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 70546104, 'displayName': 'King Gizzard...",0.030854,"{'date': '2019-08-20', 'datetime': '2019-08-20...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,King Gizzard & The Lizard Wizard,Stonefield,2019-08-20 19:00:00-06:00
3,3,"DJ Pauly D at The Depot (August 24, 2019)",False,38989431,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73852505, 'displayName': 'DJ Pauly D',...",0.009071,"{'date': '2019-08-24', 'datetime': '2019-08-24...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,DJ Pauly D,NaN,2019-08-24 21:00:00-06:00
4,4,"Corduroy at The Depot (August 31, 2019)",False,38991753,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73856755, 'displayName': 'Corduroy', '...",0.001657,"{'date': '2019-08-31', 'datetime': '2019-08-31...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Corduroy,NaN,2019-08-31 20:00:00-06:00
5,5,Grace VanderWaal and Patrick Martin at The Dep...,False,38793989,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73512384, 'displayName': 'Grace Vander...",0.024689,"{'date': '2019-09-01', 'datetime': '2019-09-01...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Grace VanderWaal,Patrick Martin,2019-09-01 18:00:00-06:00
6,6,"Die Antwoord at The Depot (September 10, 2019)",False,38081309,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 72285109, 'displayName': 'Die Antwoord...",0.073583,"{'date': '2019-09-10', 'datetime': '2019-09-10...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Die Antwoord,NaN,2019-09-10 19:00:00-06:00
7,7,"Fatboy SSE at The Depot (September 14, 2019)",False,39029270,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73918932, 'displayName': 'Fatboy SSE',...",0.000869,"{'date': '2019-09-14', 'datetime': '2019-09-14...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Fatboy SSE,NaN,2019-09-14 20:00:00-06:00
8,8,"Andy Grammer at The Depot (September 17, 2019)",False,38979297,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73834665, 'displayName': 'Andy Grammer...",0.094384,"{'date': '2019-09-17', 'datetime': '2019-09-17...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Andy Grammer,NaN,2019-09-17 19:00:00-06:00
9,9,"Oliver Tree at The Depot (September 18, 2019)",False,38735629,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73410574, 'displayName': 'Oliver Tree'...",0.014342,"{'date': '2019-09-18', 'datetime': '2019-09-18...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,Oliver Tree,NaN,2019-09-18 07:00:00-06:00


In [11]:
df.to_csv('sk_events.csv')

In [39]:
df

,Unnamed: 0,displayName,flaggedAsEnded,id,location,performance,popularity,start,type,venue,venue_id,headliner,support,event_time
0,0,Reverend Horton Heat with The Delta Bombers an...,False,37492754,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 71255334, 'displayName': 'Reverend Hor...",0.013307,"{'date': '2019-08-08', 'datetime': '2019-08-08...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-08-08 19:00:00-06:00
1,1,"The Anchorage, Racist Kramer, Problem Daughter...",False,38827049,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73571014, 'displayName': 'The Anchorag...",0.000095,"{'date': '2019-08-10', 'datetime': '2019-08-10...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-08-10 19:00:00-06:00
2,2,King Gizzard & The Lizard Wizard and Stonefiel...,False,37091759,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 70546104, 'displayName': 'King Gizzard...",0.030854,"{'date': '2019-08-20', 'datetime': '2019-08-20...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-08-20 19:00:00-06:00
3,3,"DJ Pauly D at The Depot (August 24, 2019)",False,38989431,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73852505, 'displayName': 'DJ Pauly D',...",0.009071,"{'date': '2019-08-24', 'datetime': '2019-08-24...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-08-24 21:00:00-06:00
4,4,"Corduroy at The Depot (August 31, 2019)",False,38991753,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73856755, 'displayName': 'Corduroy', '...",0.001657,"{'date': '2019-08-31', 'datetime': '2019-08-31...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-08-31 20:00:00-06:00
5,5,Grace VanderWaal and Patrick Martin at The Dep...,False,38793989,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73512384, 'displayName': 'Grace Vander...",0.024689,"{'date': '2019-09-01', 'datetime': '2019-09-01...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-09-01 18:00:00-06:00
6,6,"Die Antwoord at The Depot (September 10, 2019)",False,38081309,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 72285109, 'displayName': 'Die Antwoord...",0.073583,"{'date': '2019-09-10', 'datetime': '2019-09-10...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-09-10 19:00:00-06:00
7,7,"Fatboy SSE at The Depot (September 14, 2019)",False,39029270,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73918932, 'displayName': 'Fatboy SSE',...",0.000869,"{'date': '2019-09-14', 'datetime': '2019-09-14...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-09-14 20:00:00-06:00
8,8,"Andy Grammer at The Depot (September 17, 2019)",False,38979297,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73834665, 'displayName': 'Andy Grammer...",0.094384,"{'date': '2019-09-17', 'datetime': '2019-09-17...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-09-17 19:00:00-06:00
9,9,"Oliver Tree at The Depot (September 18, 2019)",False,38735629,"{'city': 'Salt Lake City, UT, US', 'lat': 40.7...","[{'id': 73410574, 'displayName': 'Oliver Tree'...",0.014342,"{'date': '2019-09-18', 'datetime': '2019-09-18...",Concert,"{'id': 983, 'displayName': 'The Depot', 'uri':...",983,NaN,NaN,2019-09-18 07:00:00-06:00


In [51]:
df.performance[0][1]

{'id': 71355484,
 'displayName': 'The Delta Bombers',
 'billing': 'support',
 'billingIndex': 2,
 'artist': {'id': 2384498,
  'displayName': 'The Delta Bombers',
  'uri': 'http://www.songkick.com/artists/2384498-delta-bombers?utm_source=58878&utm_medium=partner',
  'identifier': [{'mbid': '5cdd3a4d-57fb-4160-91e5-c7b95d5b4577',
    'href': 'http://api.songkick.com/api/3.0/artists/mbid:5cdd3a4d-57fb-4160-91e5-c7b95d5b4577.json'}]}}

In [32]:
venue_df = pd.read_csv('sk_venues.csv')

,Unnamed: 0,venue_name,id,capacity,street,zip,latitude,longitude,metro_area
0,0,The Basement East,2913113,NaN,917 Woodland Ave.,37206,36.175636,-86.755652,Nashville
1,1,The Basement East,4147909,545.0,917 Woodland Street,37206,36.175670,-86.755580,Nashville
2,2,The Zimmerman's Basement O' Sound,1766373,NaN,7 Sussex Road,08816,40.418433,-74.432227,East Brunswick
3,3,The Basement East,3433549,NaN,1245 Glenwood Avenue Southeast,30316,33.739988,-84.346676,Atlanta
4,4,Evangeline Downs Racetrack & Casino,2371734,NaN,2235 Creswell Lane Extension,70570,30.533530,-92.081510,Baton Rouge
5,5,Zappo's Theater at Planet Hollywood,3815719,4600.0,3667 S. Las Vegas Blvd.,89109,36.110060,-115.170890,Las Vegas
6,6,Resorts World Catskills,3894369,NaN,888 Resorts World Drive,12701,41.657150,-74.647720,Monticello
7,7,Santander Arena,506116,NaN,700 Penn St.,19602,40.335030,-75.923390,Wilmington
8,8,"Eagle Theater, Santander Arena",6896,NaN,700 Penn St.,19610,40.335140,-75.923400,Wilmington
9,9,The Theater at Santander Arena,3842839,NaN,700 Penn Street,19602,40.335030,-75.923390,Wilmington
